In [1]:

import json
import shutil
import os
import pickle
from callback import MultipleClassAUROC, MultiGPUModelCheckpoint
from configparser import ConfigParser
from generator import AugmentedImageSequence
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import multi_gpu_model
from utility import get_sample_counts
from weights import get_class_weights
from augmenter import augmenter
from tensorflow.keras import backend as K
import pandas as pd
import numpy as np
import time
import tensorflow.keras.initializers
import statistics
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, InputLayer, Flatten, Input, GaussianNoise
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from keras_radam import RAdam
from sklearn.metrics import confusion_matrix

%load_ext tensorboard

from datetime import datetime
from packaging import version

import tensorflow as tf
from tensorflow import keras

import numpy as np

#print("TensorFlow version: ", tf.__version__)
#assert version.parse(tf.__version__).release[0] >= 2, \
#    "This notebook requires TensorFlow 2.0 or above."


Using TensorFlow backend.


In [2]:
import gc

In [3]:
import random 
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
# UPDATED: import from tensorflow.keras instead of keras
from tensorflow.keras import layers, optimizers, losses, metrics
import gc
import tensorflow_addons as tfa

In [4]:
oneClass = "Atelectasis"

In [5]:
config_file = "./config.ini"
cp = ConfigParser()
cp.read(config_file)

    # default config
output_dir = cp["DEFAULT"].get("output_dir")
image_source_dir = cp["DEFAULT"].get("image_source_dir")
base_model_name = cp["DEFAULT"].get("base_model_name")
class_names = cp["DEFAULT"].get("class_names").split(",")

    # train config
use_base_model_weights = cp["TRAIN"].getboolean("use_base_model_weights")
use_trained_model_weights = cp["TRAIN"].getboolean("use_trained_model_weights")
use_best_weights = cp["TRAIN"].getboolean("use_best_weights")
output_weights_name = cp["TRAIN"].get("output_weights_name")
epochs = cp["TRAIN"].getint("epochs")
batch_size = cp["TRAIN"].getint("batch_size")
initial_learning_rate = cp["TRAIN"].getfloat("initial_learning_rate")
generator_workers = cp["TRAIN"].getint("generator_workers")
image_dimension = cp["TRAIN"].getint("image_dimension")
train_steps = cp["TRAIN"].get("train_steps")
patience_reduce_lr = cp["TRAIN"].getint("patience_reduce_lr")
min_lr = cp["TRAIN"].getfloat("min_lr")
validation_steps = cp["TRAIN"].get("validation_steps")
positive_weights_multiply = cp["TRAIN"].getfloat("positive_weights_multiply")
dataset_csv_dir = cp["TRAIN"].get("dataset_csv_dir")

In [6]:
def focal_loss(gamma=1.0, alpha=0.5):
    gamma = float(gamma)
    alpha = float(alpha)
    def focal_loss_fixed(y_true, y_pred):
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1.0-epsilon)
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
        return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1))-K.sum((1-alpha) * K.pow( pt_0, gamma) * K.log(1. - pt_0))
    return focal_loss_fixed

In [7]:
from tensorflow.keras.losses import Huber

In [8]:
def euclidean_distance_loss(y_true, y_pred):
    return K.sqrt(K.sum(K.square(tf.cast(y_pred,tf.float32) - tf.cast(y_true,tf.float32)), axis=-1))

In [9]:
def construct_network1(dropout=0.08425517073874295, neuronPct=0.1767547775828121, neuronShrink=0.33180474398878285):
    # We start with some percent of 5000 starting neurons on the first hidden layer.
    neuronCount = int(neuronPct * 5000)
    # Construct neural network
    neuronCount = neuronCount * neuronShrink
    model = Sequential()
    model.add(Input(shape=(1,1536)))
    model.add(Flatten(name='flat1'))
    model.add(Dense(neuronCount,name='dense1'))
    model.add(Activation('relu',name='relu1'))
    model.add(Dropout(dropout, name='dropout1'))
    model.add(Dense(14, activation='sigmoid',name='midLayer1')) # Output
    weights_path= None
    if weights_path is not None:
        print(f"load model weights_path: {weights_path}")
        model.load_weights(weights_path)
    model.layers.pop()
    dr = model.layers[-2].output
    model.trainable = False
    left = Dense(14, activation="sigmoid", name='leftLayer1')(dr)
    right = Dense(14, activation="sigmoid", name='rightLayer1')(dr)
    model = Model(model.input, [left,model.output,right])
    #model = Model(model.input, model.output)
    return model

In [10]:
def construct_network2(dropout=0.15672137551441198, neuronPct=0.2197894476507525, neuronShrink=0.3803316528497302, noisePct=0.282563134185142):
    # We start with some percent of 5000 starting neurons on the first hidden layer.
    neuronCount = int(neuronPct * 5000)
    # Construct neural network
    neuronCount = neuronCount * neuronShrink
    model = Sequential()
    model.add(Input(shape=(1,1536)))
    model.add(Flatten(name='flat2'))
    model.add(Dense(neuronCount,name='dense2'))
    model.add(GaussianNoise(noisePct))
    model.add(Activation('relu',name='relu2'))
    model.add(Dropout(dropout, name='dropout2'))
    model.add(Dense(14, activation='sigmoid',name='midLayer2')) # Output
    weights_path= None
    if weights_path is not None:
        print(f"load model weights_path: {weights_path}")
        model.load_weights(weights_path)
    #model.layers.pop()
    dr = model.layers[-2].output
    model.trainable = False
    left = Dense(14, activation="sigmoid", name='leftLayer2')(dr)
    right = Dense(14, activation="sigmoid", name='rightLayer2')(dr)
    model = Model(model.input, [left,model.output,right])
    #model = Model(model.input, model.output)
    return model

In [11]:
def construct_network(model1,model2):
    model = Model([model1.input,model2.input], [model1.output,model2.output])
    return model 

In [12]:
# compute steps
train_counts, train_pos_counts = get_sample_counts(output_dir, "train"+oneClass, class_names)
dev_counts, _ = get_sample_counts(output_dir, "dev"+oneClass, class_names)
    
if train_steps == "auto":
    train_steps = int(train_counts / batch_size)
else:
    try:
        train_steps = int(train_steps)
    except ValueError:
        raise ValueError(f"""train_steps: {train_steps} is invalid,please use 'auto' or integer.""")
    print(f"** train_steps: {train_steps} **")

if validation_steps == "auto":
    validation_steps = int(dev_counts / batch_size)
else:
    try:
        validation_steps = int(validation_steps)
    except ValueError:
        raise ValueError(f"""validation_steps: {validation_steps} is invalid,please use 'auto' or integer.""")
        print(f"** validation_steps: {validation_steps} **")

        # compute class weights
keras.backend.clear_session()
print("** compute class weights from training data **")
class_weights = get_class_weights(train_counts,train_pos_counts,multiply=positive_weights_multiply,)
print("** class_weights **")
print(class_weights)
#print(str(train_steps))
#print(str(train_counts))
#print(str(batch_size))

** compute class weights from training data **
7996: 7996
264: 7996
2226: 7996
2281: 7996
503: 7996
405: 7996
188: 7996
522: 7996
813: 7996
163: 7996
316: 7996
161: 7996
323: 7996
23: 7996
** class_weights **
[{0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}]


In [13]:
test_steps = cp["TEST"].get("test_steps")
test_counts, _ = get_sample_counts(output_dir, "test", class_names)

if test_steps == "auto":
    test_steps = int(test_counts / batch_size)
else:
    try:
        test_steps = int(test_steps)
    except ValueError:
        raise ValueError(f"""test_steps: {test_steps} is invalid,please use 'auto' or integer.""")
        
print(f"** test_steps: {test_steps} **")

** test_steps: 22433 **


In [14]:
train_sequence = AugmentedImageSequence(
            dataset_csv_file=os.path.join(output_dir, "train"+oneClass+".csv"),
            class_names=class_names,
            source_image_dir=image_source_dir,
            batch_size=batch_size,
            target_size=(image_dimension, image_dimension),
            augmenter=augmenter,
            steps=train_steps,
        )
validation_sequence = AugmentedImageSequence(
            dataset_csv_file=os.path.join(output_dir, "dev"+oneClass+".csv"),
            class_names=class_names,
            source_image_dir=image_source_dir,
            batch_size=batch_size,
            target_size=(image_dimension, image_dimension),
            augmenter=augmenter,
            steps=validation_steps,
            shuffle_on_epoch_end=False,
)

test_sequence = AugmentedImageSequence(
        dataset_csv_file=os.path.join(output_dir, "test.csv"),
        class_names=class_names,
        source_image_dir=image_source_dir,
        batch_size=batch_size,
        target_size=(image_dimension, image_dimension),
        augmenter=None,
        steps=test_steps,
        shuffle_on_epoch_end=False,
    )

In [15]:
def optimize_network(lr):
    gc.collect()
      # Define the Keras TensorBoard callback.
    logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")

    start_time = time.time()
    model1 = construct_network1()
    model2 = construct_network2()
    
    optimizer = SGD(lr=initial_learning_rate)
    
    alpha = 0.9340456763831478
    gamma = 1.4195808780694898
    model1.compile(optimizer=optimizer,loss={'leftLayer1':tf.keras.losses.Huber(),'midLayer1':focal_loss(gamma=gamma,alpha=alpha),'rightLayer1':euclidean_distance_loss})

    alpha = 0.7297456293468533
    gamma = 1.2700405014991505
    model2.compile(optimizer=optimizer,loss={'leftLayer2':tf.keras.losses.Huber(),'midLayer2':focal_loss(gamma=gamma,alpha=alpha),'rightLayer2':euclidean_distance_loss})
  
    model = construct_network(model1=model1,model2=model2)
    model.compile(optimizer=optimizer,loss={'leftLayer1':tf.keras.losses.Huber(),'midLayer1':focal_loss(gamma=gamma,alpha=alpha),'rightLayer1':euclidean_distance_loss,'leftLayer2':tf.keras.losses.Huber(),'midLayer2':focal_loss(gamma=gamma,alpha=alpha),'rightLayer2':euclidean_distance_loss})

    output_weights_path = os.path.join(output_dir,  str(lr)+"_"+output_weights_name)
    
    print(f"** set output weights path to: {output_weights_path} **")
                  
    
                  
    checkpoint = ModelCheckpoint(
                 output_weights_path,
                 save_weights_only=True,
                 save_best_only=True,
                 verbose=1,
            )
    start_time = time.time()
  
    model.summary()
  
    callbacks = [
            checkpoint,
            #keras.callbacks.TensorBoard(log_dir=logdir),
            #TensorBoard(log_dir=os.path.join(output_dir, "logs"), batch_size=batch_size),
            ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=patience_reduce_lr,
                              verbose=1, mode="min", min_lr=min_lr), 
            EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto', restore_best_weights=True)
    ]
    
    
    history = model.fit_generator(
            generator=train_sequence,
            steps_per_epoch=train_steps,
            epochs=epochs,
            validation_data=validation_sequence,
            validation_steps=validation_steps,
            callbacks=callbacks,
            class_weight=[class_weights,class_weights,class_weights,class_weights,class_weights,class_weights],
            workers=generator_workers,
            shuffle=False,
        )
        
    y_hat = model.predict_generator(test_sequence, verbose=1)
    y = test_sequence.get_y_true()
    
    test_log_path = os.path.join(output_dir, str(lr)+"_"+"test.log")
    print(f"** write log to {test_log_path} **")
    aurocs = []
    auprcs = []
    precision = dict()
    recall = dict()
    threshold = dict()
    with open(test_log_path, "w") as f:
        for k in range(6):
            for i in range(len(class_names)):
                 if(class_names[i] == str(oneClass)):
                
                    try:
                        score = roc_auc_score(y[:, i], y_hat[k][:, i])
                        precision[i], recall[i], threshold[i] = precision_recall_curve(y[:, i], y_hat[k][:, i])
                        tmp = auc(recall[i], precision[i])
                        aurocs.append(score)
                        auprcs.append(tmp) 
                    except ValueError:
                        score = 0
               
                    print(f"auroc {str(k)+class_names[i]}: {score}\n")
                    print(f"auprc {str(k)+class_names[i]}: {tmp}\n")
                    f.write(f"auroc {str(k)+class_names[i]}: {score}\n")
                    f.write(f"auprc {str(k)+class_names[i]}: {tmp}\n")
        
        mean_auroc = np.mean(aurocs)
        mean_auprc = float(np.mean(auprcs))
        f.write("-------------------------\n")
        f.write(f"mean auroc: {mean_auroc}\n")
        print(f"mean auroc: {mean_auroc}\n")
        f.write(f"mean auprc: {mean_auprc}\n")
        print(f"mean auprc: {mean_auprc}\n")
        
        max_auroc = np.max(aurocs)
        max_auprc = float(np.max(auprcs))
        f.write("-------------------------\n")
        f.write(f"max auroc: {max_auroc}\n")
        print(f"max auroc: {max_auroc}\n")
        f.write(f"max auprc: {max_auprc}\n")
        print(f"max auprc: {max_auprc}\n")
    
    keras.backend.clear_session()
    time_took = time.time() - start_time
    
    return max_auroc, time_took
    


In [16]:
step = np.arange(0.009, 0.0151, 0.001)
maxi = []
for k in np.nditer(step):
    opn, daTime = optimize_network(k)
    print(daTime)
    maxi.append(opn)




** set output weights path to: ./experiments/0.009_weights.h5 **
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 1, 1536)]    0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 1, 1536)]    0                                            
__________________________________________________________________________________________________
flat2 (Flatten)                 (None, 1536)         0           input_2[0][0]                    
__________________________________________________________________________________________________
flat1 (Flatten)                 (None, 1536)         0           input_1[0][0]                    
___________________________

Epoch 6/11
7987/7996 [============================>.] - ETA: 0s - loss: 4.6327 - leftLayer1_loss: 0.0633 - midLayer1_loss: 1.4423 - rightLayer1_loss: 0.8240 - leftLayer2_loss: 0.0405 - midLayer2_loss: 1.4716 - rightLayer2_loss: 0.7911
Epoch 00006: val_loss improved from 4.57255 to 4.53509, saving model to ./experiments/0.009_weights.h5
7996/7996 [==============================] - 20s 3ms/step - loss: 4.6321 - leftLayer1_loss: 0.0633 - midLayer1_loss: 1.4422 - rightLayer1_loss: 0.8237 - leftLayer2_loss: 0.0405 - midLayer2_loss: 1.4715 - rightLayer2_loss: 0.7908 - val_loss: 4.5351 - val_leftLayer1_loss: 0.0603 - val_midLayer1_loss: 1.4317 - val_rightLayer1_loss: 0.7906 - val_leftLayer2_loss: 0.0561 - val_midLayer2_loss: 1.3520 - val_rightLayer2_loss: 0.8443
Epoch 7/11
7977/7996 [============================>.] - ETA: 0s - loss: 4.6109 - leftLayer1_loss: 0.0588 - midLayer1_loss: 1.4419 - rightLayer1_loss: 0.8124 - leftLayer2_loss: 0.0385 - midLayer2_loss: 1.4723 - rightLayer2_loss: 0.7871

  {'leftLayer1': '...', 'midLayer1': '...', 'rightLayer1': '...', 'leftLayer2': '...', 'midLayer2': '...', 'rightLayer2': '...'}
    to  
  ['...', '...', '...', '...', '...', '...']
Train for 7996 steps, validate for 1119 steps
Epoch 1/11
7979/7996 [============================>.] - ETA: 0s - loss: 5.5507 - leftLayer1_loss: 0.1181 - midLayer1_loss: 1.4091 - rightLayer1_loss: 1.4232 - leftLayer2_loss: 0.1009 - midLayer2_loss: 1.3178 - rightLayer2_loss: 1.1817
Epoch 00001: val_loss improved from inf to 5.11639, saving model to ./experiments/0.009999999999999998_weights.h5
7996/7996 [==============================] - 21s 3ms/step - loss: 5.5492 - leftLayer1_loss: 0.1181 - midLayer1_loss: 1.4091 - rightLayer1_loss: 1.4225 - leftLayer2_loss: 0.1008 - midLayer2_loss: 1.3177 - rightLayer2_loss: 1.1810 - val_loss: 5.1164 - val_leftLayer1_loss: 0.1086 - val_midLayer1_loss: 1.3939 - val_rightLayer1_loss: 1.1135 - val_leftLayer2_loss: 0.0976 - val_midLayer2_loss: 1.2851 - val_rightLayer2_loss: 1

Epoch 4/11
7984/7996 [============================>.] - ETA: 0s - loss: 4.4671 - leftLayer1_loss: 0.0764 - midLayer1_loss: 1.3204 - rightLayer1_loss: 0.8470 - leftLayer2_loss: 0.0472 - midLayer2_loss: 1.3721 - rightLayer2_loss: 0.8041
Epoch 00004: val_loss improved from 4.52266 to 4.44738, saving model to ./experiments/0.010999999999999998_weights.h5
7996/7996 [==============================] - 20s 3ms/step - loss: 4.4665 - leftLayer1_loss: 0.0764 - midLayer1_loss: 1.3203 - rightLayer1_loss: 0.8468 - leftLayer2_loss: 0.0472 - midLayer2_loss: 1.3720 - rightLayer2_loss: 0.8039 - val_loss: 4.4474 - val_leftLayer1_loss: 0.0714 - val_midLayer1_loss: 1.3096 - val_rightLayer1_loss: 0.8077 - val_leftLayer2_loss: 0.0613 - val_midLayer2_loss: 1.3226 - val_rightLayer2_loss: 0.8748
Epoch 5/11
7987/7996 [============================>.] - ETA: 0s - loss: 4.4256 - leftLayer1_loss: 0.0684 - midLayer1_loss: 1.3204 - rightLayer1_loss: 0.8243 - leftLayer2_loss: 0.0428 - midLayer2_loss: 1.3738 - rightLaye

  {'leftLayer1': '...', 'midLayer1': '...', 'rightLayer1': '...', 'leftLayer2': '...', 'midLayer2': '...', 'rightLayer2': '...'}
    to  
  ['...', '...', '...', '...', '...', '...']
Train for 7996 steps, validate for 1119 steps
Epoch 1/11
7975/7996 [============================>.] - ETA: 0s - loss: 5.5864 - leftLayer1_loss: 0.1164 - midLayer1_loss: 1.3733 - rightLayer1_loss: 1.4235 - leftLayer2_loss: 0.1090 - midLayer2_loss: 1.4308 - rightLayer2_loss: 1.1334
Epoch 00001: val_loss improved from inf to 5.08257, saving model to ./experiments/0.011999999999999997_weights.h5
7996/7996 [==============================] - 20s 3ms/step - loss: 5.5845 - leftLayer1_loss: 0.1164 - midLayer1_loss: 1.3732 - rightLayer1_loss: 1.4226 - leftLayer2_loss: 0.1090 - midLayer2_loss: 1.4308 - rightLayer2_loss: 1.1326 - val_loss: 5.0826 - val_leftLayer1_loss: 0.1072 - val_midLayer1_loss: 1.3576 - val_rightLayer1_loss: 1.1115 - val_leftLayer2_loss: 0.0981 - val_midLayer2_loss: 1.3463 - val_rightLayer2_loss: 1

Epoch 4/11
7976/7996 [============================>.] - ETA: 0s - loss: 4.5440 - leftLayer1_loss: 0.0766 - midLayer1_loss: 1.3538 - rightLayer1_loss: 0.8544 - leftLayer2_loss: 0.0498 - midLayer2_loss: 1.4089 - rightLayer2_loss: 0.8004
Epoch 00004: val_loss improved from 4.55806 to 4.47850, saving model to ./experiments/0.012999999999999996_weights.h5
7996/7996 [==============================] - 20s 2ms/step - loss: 4.5435 - leftLayer1_loss: 0.0766 - midLayer1_loss: 1.3537 - rightLayer1_loss: 0.8543 - leftLayer2_loss: 0.0498 - midLayer2_loss: 1.4089 - rightLayer2_loss: 0.8002 - val_loss: 4.4785 - val_leftLayer1_loss: 0.0718 - val_midLayer1_loss: 1.3420 - val_rightLayer1_loss: 0.8146 - val_leftLayer2_loss: 0.0652 - val_midLayer2_loss: 1.3015 - val_rightLayer2_loss: 0.8835
Epoch 5/11
7980/7996 [============================>.] - ETA: 0s - loss: 4.4955 - leftLayer1_loss: 0.0690 - midLayer1_loss: 1.3536 - rightLayer1_loss: 0.8298 - leftLayer2_loss: 0.0445 - midLayer2_loss: 1.4059 - rightLaye

  {'leftLayer1': '...', 'midLayer1': '...', 'rightLayer1': '...', 'leftLayer2': '...', 'midLayer2': '...', 'rightLayer2': '...'}
    to  
  ['...', '...', '...', '...', '...', '...']
Train for 7996 steps, validate for 1119 steps
Epoch 1/11
7992/7996 [============================>.] - ETA: 0s - loss: 5.8526 - leftLayer1_loss: 0.1158 - midLayer1_loss: 1.3588 - rightLayer1_loss: 1.4564 - leftLayer2_loss: 0.1070 - midLayer2_loss: 1.6626 - rightLayer2_loss: 1.1519
Epoch 00001: val_loss improved from inf to 5.27027, saving model to ./experiments/0.013999999999999995_weights.h5
7996/7996 [==============================] - 20s 3ms/step - loss: 5.8521 - leftLayer1_loss: 0.1158 - midLayer1_loss: 1.3588 - rightLayer1_loss: 1.4562 - leftLayer2_loss: 0.1070 - midLayer2_loss: 1.6625 - rightLayer2_loss: 1.1517 - val_loss: 5.2703 - val_leftLayer1_loss: 0.1071 - val_midLayer1_loss: 1.3461 - val_rightLayer1_loss: 1.1395 - val_leftLayer2_loss: 0.0966 - val_midLayer2_loss: 1.4816 - val_rightLayer2_loss: 1

Epoch 4/11
7974/7996 [============================>.] - ETA: 0s - loss: 4.5624 - leftLayer1_loss: 0.0789 - midLayer1_loss: 1.4055 - rightLayer1_loss: 0.8526 - leftLayer2_loss: 0.0486 - midLayer2_loss: 1.3745 - rightLayer2_loss: 0.8023
Epoch 00004: val_loss improved from 4.64113 to 4.56317, saving model to ./experiments/0.014999999999999994_weights.h5
7996/7996 [==============================] - 20s 3ms/step - loss: 4.5620 - leftLayer1_loss: 0.0789 - midLayer1_loss: 1.4055 - rightLayer1_loss: 0.8524 - leftLayer2_loss: 0.0486 - midLayer2_loss: 1.3745 - rightLayer2_loss: 0.8021 - val_loss: 4.5632 - val_leftLayer1_loss: 0.0740 - val_midLayer1_loss: 1.4008 - val_rightLayer1_loss: 0.8156 - val_leftLayer2_loss: 0.0635 - val_midLayer2_loss: 1.3358 - val_rightLayer2_loss: 0.8735
Epoch 5/11
7977/7996 [============================>.] - ETA: 0s - loss: 4.5170 - leftLayer1_loss: 0.0705 - midLayer1_loss: 1.4053 - rightLayer1_loss: 0.8289 - leftLayer2_loss: 0.0438 - midLayer2_loss: 1.3744 - rightLaye

In [17]:
print(np.max(maxi))

0.6824147155588537
